# Imports

In [2]:
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv
from matplotlib import pyplot as plt
import seaborn as sns

# Reading and cleaning Google Places dataframe

In [81]:
# Reading the Toronto hotels dataset (from Google Places API)
df_places = pd.read_csv('hotels_in_toronto.csv')
df_places.head(3)

,Unnamed: 0,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,...,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,plus_code.compound_code,plus_code.global_code,opening_hours.open_now
0,0,OPERATIONAL,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Hilton Toronto,"[{'height': 833, 'html_attributions': ['<a hre...",ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,...,4747,43.649852,-79.385576,43.651296,-79.384380,43.648596,-79.387080,"JJX7+WQ Toronto, Ontario",87M2JJX7+WQ,NaN
1,1,OPERATIONAL,"1 King St W, Toronto, ON M5H 1A1, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,One King West Hotel & Residence,"[{'height': 4931, 'html_attributions': ['<a hr...",ChIJG74DgzLL1IkRu7-_ro1TbdY,4.4,...,4771,43.648873,-79.378072,43.650154,-79.376654,43.647454,-79.379354,"JJXC+GQ Toronto, Ontario",87M2JJXC+GQ,True
2,2,OPERATIONAL,"7095 Woodbine Ave, Markham, ON L3R 1A3, Canada",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Courtyard by Marriott Toronto Northeast/Markham,"[{'height': 320, 'html_attributions': ['<a hre...",ChIJVy3BfGXT1IkRjZhm3zfseUE,4.2,...,1438,43.816659,-79.348556,43.818135,-79.347102,43.815435,-79.349801,"RM82+MH Markham, Ontario",87M2RM82+MH,NaN


In [82]:
# Dropping unneeded columns
df_places = df_places[['formatted_address', 'name',
       'place_id', 'rating', 'types', 'user_ratings_total',
       'geometry.location.lat', 'geometry.location.lng',
       'opening_hours.open_now']]

# Prefacing column names with "places"
df_places.columns = ['places_formatted_address', 'places_name',
       'places_id', 'places_rating', 'places_types', 'places_user_ratings_total',
       'places_lat', 'places_lng',
       'places_opening_hours.open_now']

# Creating function for calling Yelp API

In [83]:
# Using dotenv to retrieve Yelp API key
load_dotenv()
api_key_y_string = 'YELP_API_KEY2' # replace with your Yelp API key variable name
api_key_y = os.getenv(api_key_y_string)

In [89]:
# Defining the function to find all POI's in Yelp within a set distance from each hotel
def find_POIs_yelp(places_ids,latitudes,longitudes,radius,limit,categories):
    
    #fields = 'fsq_id,name,rating,popularity,price'
    list_POI = []
    
    # Find all POI's within specified radius of each bike station.  This will return one column with the place_id and one column containing lists of all POI's.
    for pl_id,lat,long in zip(places_ids,latitudes,longitudes):
        url = f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius={radius}&limit={limit}&categories={categories}'
        headers = {"accept": "application/json","Authorization": 'bearer ' + api_key_y}
        response = requests.get(url, headers=headers).json()['businesses']
        list_POI.append([pl_id,response])
    
    # convert to dataframe
    df = pd.DataFrame(list_POI,columns=['places_id','businesses'])

    # create 1 row for each match of bike station to POI.  There are duplicates of both bike stations and POI's, but each matchup is unique.
    df = df.explode('businesses').reset_index(drop=True) 
    df = pd.concat([df,pd.json_normalize(df['businesses'])],axis=1).drop('businesses',axis=1)
    return df

# Running function to pull nearby POI's for each hotel

In [98]:
# Set up parameters to run find_POIs_yelp() function

n = 2 # number of hotels to be considered (first n rows of df_places)
r = 200 # search radius in metres
lim = 20 # limit of POI's returned for each hotel
cat = 'Food (food, All)' # Yelp POI category

df_test = df_places.head(n) # create new dataframe with first n hotels

# Create dataframe of every unique matchup of hotel to POI
df_yelp = find_POIs_yelp(df_test['places_id'],df_test['places_lat'],df_test['places_lng'],r,lim,cat)

In [112]:
df_yelp.head(3)

,places_id,yelp_id,yelp_name,yelp_review_count,yelp_categories,yelp_rating,yelp_price,yelp_latitude,yelp_longitude,yelp_display_address
0,ChIJJd9P-800K4gRvTdTqtNCmQw,C6vH0qzmcerMkiJy3VO9qA,Estiatorio Volos,334,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,$$$,43.650160,-79.384770,"[133 Richmond St W, Toronto, ON M5H 2L3, Canada]"
1,ChIJJd9P-800K4gRvTdTqtNCmQw,ro82qYK_AETB8wNGA5Xdzw,Kōjin,109,"[{'alias': 'newcanadian', 'title': 'Canadian (...",4.0,$$$,43.649330,-79.386398,"[190 University Ave, Toronto, ON M5H 0A3, Canada]"
2,ChIJJd9P-800K4gRvTdTqtNCmQw,Q2ZNaN3p8s_-XXjBWyY2qA,Ruth's Chris Steak House,242,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,$$$$,43.650052,-79.385742,"[145 Richmond Street W, Toronto, ON M5H 2L2, C..."


In [114]:
df_merge = df_test.merge(df_yelp,on='places_id',how='left')
df_merge.head(3)

,places_formatted_address,places_name,places_id,places_rating,places_types,places_user_ratings_total,places_lat,places_lng,places_opening_hours.open_now,yelp_id,yelp_name,yelp_review_count,yelp_categories,yelp_rating,yelp_price,yelp_latitude,yelp_longitude,yelp_display_address
0,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,C6vH0qzmcerMkiJy3VO9qA,Estiatorio Volos,334,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,$$$,43.650160,-79.384770,"[133 Richmond St W, Toronto, ON M5H 2L3, Canada]"
1,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,ro82qYK_AETB8wNGA5Xdzw,Kōjin,109,"[{'alias': 'newcanadian', 'title': 'Canadian (...",4.0,$$$,43.649330,-79.386398,"[190 University Ave, Toronto, ON M5H 0A3, Canada]"
2,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,Q2ZNaN3p8s_-XXjBWyY2qA,Ruth's Chris Steak House,242,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,$$$$,43.650052,-79.385742,"[145 Richmond Street W, Toronto, ON M5H 2L2, C..."


In [105]:
# Convert yelp_price to int 
df_merge['yelp_price_int'] = df_merge['yelp_price'].fillna('').str.len().astype(int)

In [106]:
df_merge.head()

,places_formatted_address,places_name,places_id,places_rating,places_types,places_user_ratings_total,places_lat,places_lng,places_opening_hours.open_now,yelp_id,yelp_name,yelp_review_count,yelp_categories,yelp_rating,yelp_price,yelp_latitude,yelp_longitude,yelp_display_address,yelp_price_int
0,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,C6vH0qzmcerMkiJy3VO9qA,Estiatorio Volos,334,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,$$$,43.650160,-79.384770,"[133 Richmond St W, Toronto, ON M5H 2L3, Canada]",3
1,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,ro82qYK_AETB8wNGA5Xdzw,Kōjin,109,"[{'alias': 'newcanadian', 'title': 'Canadian (...",4.0,$$$,43.649330,-79.386398,"[190 University Ave, Toronto, ON M5H 0A3, Canada]",3
2,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,Q2ZNaN3p8s_-XXjBWyY2qA,Ruth's Chris Steak House,242,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,$$$$,43.650052,-79.385742,"[145 Richmond Street W, Toronto, ON M5H 2L2, C...",4
3,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,4-FgbYd8jUSqevNRxRtS-w,Rosalinda,193,"[{'alias': 'vegan', 'title': 'Vegan'}]",4.0,$$,43.650160,-79.384770,"[133 Richmond Street W, Toronto, ON M5H 2L3, C...",2
4,"145 Richmond St W, Toronto, ON M5H 2L2, Canada",Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,"['lodging', 'point_of_interest', 'establishment']",4747,43.649852,-79.385576,NaN,ocLnCE2E29j-CoBQj1yaSA,Cafe Landwer - Adelaide & University,516,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.0,$$,43.648750,-79.384910,"[165 University Avenue, Toronto, ON M5H 3B8, C...",2


# Calculating average POI metrics for each hotel

In [111]:
# Create pivot table to calculate average 'yelp_review_count','yelp_rating', and 'yelp_price_int' of nearby restaurants for each hotel
pivot_merge = df_merge.pivot_table(index=['places_name','places_id'],values=['places_rating','places_user_ratings_total','yelp_review_count','yelp_rating','yelp_price_int'],
                     aggfunc='mean')
pivot_merge

,,places_rating,places_user_ratings_total,yelp_price_int,yelp_rating,yelp_review_count
places_name,places_id,,,,,
Hilton Toronto,ChIJJd9P-800K4gRvTdTqtNCmQw,4.2,4747,2.3,3.825,179.5
One King West Hotel & Residence,ChIJG74DgzLL1IkRu7-_ro1TbdY,4.4,4771,2.2,3.650,145.2


# Exploratory Data Analysis